# 673. Number of Longest Increasing Subsequence
#### https://leetcode.com/problems/number-of-longest-increasing-subsequence/

# Approach 1: Dynamic Programming

In [44]:
class Solution(object):
    def findNumberOfLIS(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if not nums: return 0
        n = len(nums)
        dp = [1] * n
        cnt = [1] * n
        res = 0
        max_len= 0
        for i in range(len(nums)):
            for j in range(i):
                if nums[j] < nums[i]:
                    if dp[j] + 1 >= dp[i]:
                        if dp[j] + 1 == dp[i]: cnt[i] += cnt[j]
                        else: cnt[i] = cnt[j]
                        dp[i] = dp[j] + 1
            
            if dp[i] >= max_len:
                #print(i,dp[i], max_len, res, cnt)
                if dp[i] == max_len: res += cnt[i]
                else: res = cnt[i]
                max_len = dp[i]
        #print(dp)        
        return res

# Approach 2: Segment Tree

In [52]:
from collections import deque
class Node(object):
    def __init__(self, start, end):
        self.range_left, self.range_right = start, end
        self._left = self._right = None
        self.val = 0, 1 #length, count
    @property
    def range_mid(self):
        return (self.range_left + self.range_right) // 2
    @property
    def left(self):
        self._left = self._left or Node(self.range_left, self.range_mid)
        return self._left
    @property
    def right(self):
        self._right = self._right or Node(self.range_mid + 1, self.range_right)
        return self._right

def merge(v1, v2):
    if v1[0] == v2[0]:
        if v1[0] == 0: return (0, 1)
        return v1[0], v1[1] + v2[1]
    return max(v1, v2)

def insert(node, key, val):
    if node.range_left == node.range_right:
        node.val = merge(val, node.val)
        return
    if key <= node.range_mid:
        insert(node.left, key, val)
    else:
        insert(node.right, key, val)
    node.val = merge(node.left.val, node.right.val)

def query(node, key):
    if node.range_right <= key:
        return node.val
    elif node.range_left > key:
        return 0, 1
    else:
        return merge(query(node.left, key), query(node.right, key))

class Solution(object):
    def findNumberOfLIS(self, nums):
        if not nums: return 0
        root = Node(min(nums), max(nums))
        for num in nums:
            length, count = query(root, num-1)
            print(num, length, count)
            insert(root, num, (length+1, count))
        queue = deque()
        queue.append(root)
        while queue:
            node = queue.popleft()
            print(node.range_left, node.range_right, node.val)
            if node._left: queue.append(node._left)
            if node._right: queue.append(node._right)
        return root.val[1]

# Verification

In [53]:
testcases = [[1,3,5,4,7],[2,2,2,2,2],[1,2,4,3,5,4,7,2]]
expected = [2,5,3]

testcases = testcases[-1:]
expected = expected[-1:]

outputs = []
sol = Solution();

for case in zip(testcases, expected):
    print("Input:    " + str(case[0]))
    print("Output:   " + str(sol.findNumberOfLIS(case[0])))
    print("Expected: " + str(case[1]) + "\n")

Input:    [1, 2, 4, 3, 5, 4, 7, 2]
1 0 1
2 1 1
4 2 1
3 2 1
5 3 2
4 3 1
7 4 3
2 1 1
1 7 (5, 3)
1 4 (4, 1)
5 7 (5, 3)
1 2 (2, 2)
3 4 (4, 1)
5 6 (4, 2)
7 7 (5, 3)
1 1 (1, 1)
2 2 (2, 2)
3 3 (3, 1)
4 4 (4, 1)
5 5 (4, 2)
6 6 (0, 1)
Output:   3
Expected: 3

